# 第七章 函数装饰器和闭包

## 7.1 装饰器基础知识

## 7.2 python 何时执行装饰器

In [1]:
# BEGIN REGISTRATION

registry = []  # <1>

def register(func):  # <2>
    print('running register(%s)' % func)  # <3>
    registry.append(func)  # <4>
    return func  # <5>

@register  # <6>
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():  # <7>
    print('running f3()')

def main():  # <8>
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

running register(<function f1 at 0x10dc5fb90>)
running register(<function f2 at 0x10dc5fe60>)


In [2]:
main()

running main()
registry -> [<function f1 at 0x10dc5fb90>, <function f2 at 0x10dc5fe60>]
running f1()
running f2()
running f3()


## 7.3 使用装饰器改进“策略”模式

In [3]:
# strategy_best4.py
# Strategy pattern -- function-based implementation
# selecting best promotion from list of functions
# registered by a decorator

from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')


class LineItem:

    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity


class Order:  # the Context

    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())

# BEGIN STRATEGY_BEST4

promos = []  # <1>

def promotion(promo_func):  # <2>
    promos.append(promo_func)
    return promo_func

@promotion  # <3>
def fidelity(order):
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * .1
    return discount

@promotion
def large_order(order):
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * .07
    return 0

def best_promo(order):  # <4>
    """Select best discount available
    """
    return max(promo(order) for promo in promos)

# END STRATEGY_BEST4


In [4]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
         LineItem('apple', 10, 1.5),
         LineItem('watermellon', 5, 5.0)]
banana_cart = [LineItem('banana', 30, .5),
                LineItem('apple', 10, 1.5)]

In [5]:
Order(joe, cart, fidelity)

<Order total: 42.00 due: 42.00>

In [6]:
Order(ann, cart, fidelity)

<Order total: 42.00 due: 39.90>

In [7]:
Order(joe, banana_cart, bulk_item)

<Order total: 30.00 due: 28.50>

In [8]:
long_order = [LineItem(str(item_code), 1, 1.0)
               for item_code in range(10)]

In [9]:
Order(joe, long_order, large_order)

<Order total: 10.00 due: 9.30>

In [10]:
Order(joe, cart, large_order)

<Order total: 42.00 due: 42.00>

In [11]:
Order(joe, long_order, best_promo)

<Order total: 10.00 due: 9.30>

In [12]:
Order(joe, banana_cart, best_promo)

<Order total: 30.00 due: 28.50>

In [13]:
Order(ann, cart, best_promo)

<Order total: 42.00 due: 39.90>

## 7.4 变量作用域规则

## 7.5 闭包

In [14]:
class Averager():
    
    def __init__(self):
        self.series = []

    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)

In [15]:
avg = Averager()
avg(10)

10.0

In [16]:
avg(11)

10.5

In [17]:
avg(12)

11.0

In [18]:
def make_averager():
    series = []

    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)

    return averager

In [19]:
avg = make_averager()
avg(10)

10.0

In [20]:
avg(11)

10.5

In [21]:
avg(12)

11.0

In [22]:
avg.__code__.co_varnames

('new_value', 'total')

In [23]:
avg.__code__.co_freevars

('series',)

In [24]:
avg.__closure__

(<cell at 0x10dcd2290: list object at 0x10db7eaa0>,)

In [25]:
avg.__closure__[0].cell_contents

[10, 11, 12]

## 7.6 nonlocal声明

In [26]:
def make_averager():
    count = 0 
    total = 0

    def averager(new_value):
        count += 1 
        total += new_value 
        return total / count

    return averager

In [27]:
avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

In [28]:
def make_averager():
    count = 0 
    total = 0

    def averager(new_value):
        nonlocal count,total
        count += 1 
        total += new_value 
        return total / count

    return averager

In [29]:
avg = make_averager()
avg(10)

10.0

In [30]:
avg(11)

10.5

In [31]:
avg(12)

11.0

## 7.7实现一个简单的装饰器

In [32]:
# 一个简单的装饰器，输出函数的运行时间
import time
def clock(func):
    def clocked(*args):
        t0 = time.time()
        result = func(*args)
        elapsed = time.time() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [33]:
@clock 
def snooze(seconds):
    time.sleep(seconds)

@clock 
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

In [34]:
print('*' * 40, 'Calling snooze(.123)') 
snooze(.123) 
print('*' * 40, 'Calling factorial(6)') 
print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12627506s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000095s] factorial(1) -> 1
[0.00009584s] factorial(2) -> 2
[0.00014424s] factorial(3) -> 6
[0.00018907s] factorial(4) -> 24
[0.00020909s] factorial(5) -> 120
[0.00023103s] factorial(6) -> 720
6! = 720


In [35]:
factorial.__name__

'clocked'

In [36]:
# clockdeco2.py

import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args,**kwargs):
        t0 = time.time()
        result = func(*args,**kwargs)
        elapsed = time.time() - t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k,w) for k,w in sorted(kwargs.items())]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

## 7.8 标准库中的装饰器

In [37]:
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

In [38]:
print(fibonacci(6))

[0.00000119s] fibonacci(0) -> 0
[0.00000119s] fibonacci(1) -> 1
[0.00008535s] fibonacci(2) -> 1
[0.00000000s] fibonacci(1) -> 1
[0.00000000s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00004196s] fibonacci(2) -> 1
[0.00008512s] fibonacci(3) -> 2
[0.00022507s] fibonacci(4) -> 3
[0.00000000s] fibonacci(1) -> 1
[0.00000095s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00004697s] fibonacci(2) -> 1
[0.00026083s] fibonacci(3) -> 2
[0.00000119s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00004125s] fibonacci(2) -> 1
[0.00000095s] fibonacci(1) -> 1
[0.00000095s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00004196s] fibonacci(2) -> 1
[0.00008202s] fibonacci(3) -> 2
[0.00016522s] fibonacci(4) -> 3
[0.00046778s] fibonacci(5) -> 5
[0.00073814s] fibonacci(6) -> 8
8


In [39]:
@functools.lru_cache() # <1>
@clock  # <2>
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

In [40]:
print(fibonacci(6))

[0.00000095s] fibonacci(0) -> 0
[0.00000000s] fibonacci(1) -> 1
[0.00008774s] fibonacci(2) -> 1
[0.00000119s] fibonacci(3) -> 2
[0.00013304s] fibonacci(4) -> 3
[0.00000095s] fibonacci(5) -> 5
[0.00017691s] fibonacci(6) -> 8
8


In [41]:
import html
def htmlize(obj):
    content = html.escape(repr(obj)) 
    return '<pre>{}</pre>'.format(content)

In [42]:
htmlize({1, 2, 3})

'<pre>{1, 2, 3}</pre>'

In [43]:
htmlize(42)

'<pre>42</pre>'

In [44]:
print(htmlize(['alpha', 66, {3, 2, 1}]))

<pre>[&#x27;alpha&#x27;, 66, {1, 2, 3}]</pre>


In [45]:
from functools import singledispatch
from collections import abc
import numbers
import html

@singledispatch  # <1>
def htmlize(obj):
    content = html.escape(repr(obj))
    return '<pre>{}</pre>'.format(content)

@htmlize.register(str)  # <2>
def _(text):            # <3>
    content = html.escape(text).replace('\n', '<br>\n')
    return '<p>{0}</p>'.format(content)

@htmlize.register(numbers.Integral)  # <4>
def _(n):
    return '<pre>{0} (0x{0:x})</pre>'.format(n)

@htmlize.register(tuple)  # <5>
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'


In [46]:
htmlize({1, 2, 3})

'<pre>{1, 2, 3}</pre>'

In [47]:
print(htmlize(['alpha', 66, {3, 2, 1}]))

<ul>
<li><p>alpha</p></li>
<li><pre>66 (0x42)</pre></li>
<li><pre>{1, 2, 3}</pre></li>
</ul>


## 7.9 叠放装饰器

## 7.10 参数化装饰器

In [48]:
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT):  # <1>
    def decorate(func):      # <2>
        def clocked(*_args): # <3>
            t0 = time.time()
            _result = func(*_args)  # <4>
            elapsed = time.time() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)  # <5>
            result = repr(_result)  # <6>
            print(fmt.format(**locals()))  # <7>
            return _result  # <8>
        return clocked  # <9>
    return decorate  # <10>

In [49]:
@clock()  # <11>
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

[0.12792015s] snooze(0.123) -> None
[0.12333417s] snooze(0.123) -> None
[0.12576294s] snooze(0.123) -> None


In [50]:
@clock('{name}: {elapsed}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze: 0.12711811065673828s
snooze: 0.123291015625s
snooze: 0.12684416770935059s


In [51]:
@clock('{name}({args}) dt={elapsed:0.3f}s')
def snooze(seconds):
    time.sleep(seconds)

for i in range(3):
    snooze(.123)

snooze(0.123) dt=0.131s
snooze(0.123) dt=0.123s
snooze(0.123) dt=0.127s
